In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import psycopg2 as psy
import numpy as np
import tqdm
from datetime import date
import csv, pyodbc

In [9]:
data = pd.read_csv("tblData.csv")#lee la base de datos
data = data.loc[data.IDDataTypeSub1 == "CALIBRATED"] #toma solo los valores calibrados

In [26]:
def radiator():
    """selecciona los datos de la base de datos y retorna el espectro y su intensidad para la ultima medicion"""

    
    datos = data.iloc[-1].Data
   
    datos = re.split('0\r\n',datos)
    corr = [re.split("\s", value)[1:-2] for value in datos]
    rad = [float(value[0]) for value in corr]
    valor = [float(value[1]) for value in corr]

    
    return (rad, valor)
 


In [29]:
espectro, intensidad = radiator()

In [28]:
def create_table():
    #connecting to the postgressql server (20 mb free)
    conn = None
    try:
        conn = psy.connect(host="salt.db.elephantsql.com", 
                           user="aeaudjwo", password="5uHoUq62gU5BoS2HvJQH0ogSw_94nxCE")

        c = conn.cursor()

        c.execute("""CREATE TABLE espectro(
                    fecha VARCHAR(255) UNIQUE,
                    espectro REAL,
                    intensidad REAL)""")
        conn.commit()
        conn.close()
        
    except (Exception, psy.DatabaseError) as error:
        print(error)
        
    finally:
        if conn is not None:
            conn.close()
            
        
create_table()


In [34]:
def insert_data(espectro,intensidad):
    conn = None
    try:
        conn = psy.connect(host="salt.db.elephantsql.com", 
                           user="aeaudjwo", password="5uHoUq62gU5BoS2HvJQH0ogSw_94nxCE")

        c = conn.cursor()
        
        c.execute("""INSERT INTO espectro (               
                    espectro,
                    intensidad) VALUES (%s,%s)""",(espectro,intensidad))
        conn.commit()
        conn.close()
        
    except (Exception, psy.DatabaseError) as error:
        print (error)
    
    finally:
        if conn is not None:
            conn.close()


In [35]:
#inserta los valores nuevos
for spc,inten in zip(espectro, intensidad):
    insert_data(spc,inten)